In [ ]:
!pip install tflite-support

import os
from google.colab import drive
import tensorflow as tf
from tensorflow import lite
from tensorflow.python import train
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tflite_support import flatbuffers
from tflite_support import metadata as _metadata
from tflite_support import metadata_schema_py_generated as _metadata_fb


!mkdir -p drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# -- Fonction d'extraction des coordonnées (x, y) depuis un fichier texte --
def extract_xy_from_textfile(path, seq_length, nb_features=24):
  with open(path) as f:
    seq = []
    for line in f:
      frame = []
      points = line.split()
      for i in range(len(points)):
        if(i+1%3 != 0):
          frame.append(float(points[i]))
        if len(frame) >= nb_features:
          break
      seq.append(frame)
      if len(seq) >= seq_length:
        break
    # Padding des séquences pour qu'elles soient toutes de même dimension
    while len(seq) < seq_length:
      padding = [0.0 for x in range(nb_features)]
      seq.append(padding)
  return seq

# -- Fonction d'extraction des coordonnées (x, y, z) depuis un fichier texte --
def extract_xyz_from_textfile(path, seq_length, nb_features=36):
  with open(path) as f:
    seq = []
    for line in f:
      frame = []
      for p in line.split():
        frame.append(float(p))
        if len(frame) >= nb_features:
          break
      seq.append(frame)
      if len(seq) >= seq_length:
        break
    # Padding des séquences pour qu'elles soient toutes de même dimension
    while len(seq) < seq_length:
      padding = [0.0 for x in range(nb_features)]
      seq.append(padding)
  return seq

# -- Fonction d'extraction des fichiers npy --
def extract_all_npy(path, seq_length, labels_list):
  data = []
  label = []
  for d in os.listdir(path):
    for e in os.listdir(path+'/'+d):
      for f in os.listdir(path+'/'+d+'/'+e):
        label.append([labels_list.index(e+d)])
        npy = np.load(path+'/'+d+'/'+e+'/'+f)
        while len(npy) < seq_length:
          padding = [0.0 for x in range(nb_features)]
          npy.append(padding)
        data.append(npy)
  return label, data


In [6]:
labels_list = ['FalseDEADLIFT','TrueDEADLIFT','TrueSQUAT','FalseSQUAT','TrueBENCH','FalseBENCH']
# -- Paramètres --
path = '/DATASET/Dataset-movement-txt'
path_npy = '/DATASET/Dataset-movement'
Data_X = []
label = []

# -- Nombre de timestep(frames) des séquences(vidéos) --
seq_length = 440

# -- Nombre de points par timestep(frame) --
nb_features = 24

for d in os.listdir(path):
  for e in os.listdir(path+'/'+d):
    for f in os.listdir(path+'/'+d+'/'+e):
      with open(path+'/'+d+'/'+e+'/'+f) as filename:
        seq = []
        for line in filename:
          frame = []
          words = line.split()
          for i in range(len(words)):
            if (i+1)%3 != 0:
              frame.append(float(words[i]))
            if len(frame) >= nb_features:
              break
          seq.append(frame)
          if len(seq) >= seq_length:
            break
        while len(seq) < seq_length:
          pad = [0.0 for x in range(nb_features)]
          seq.append(pad)
      label.append([labels_list.index(e+d)])
      Data_X.append(seq)

In [48]:
# -- Split test/train --
xTrain, xTest, yTrain, yTest = train_test_split(
    Data_X, label, test_size=0.3, random_state=42)

xTrain = tf.convert_to_tensor(xTrain) # Données d'entrainement
xTest = tf.convert_to_tensor(xTest) # Données de validation
yTrain = tf.convert_to_tensor(yTrain) # Labels des données d'entrainement
yTest = tf.convert_to_tensor(yTest) # Labels des données de validation

# -- Hyperparamétres --
verbose = 1
epochs = 50
batch_size = 10

# -- Architecture du réseau LSTM --
model_lstm = tf.keras.models.Sequential([
    tf.keras.layers.Masking(mask_value=0.0, input_shape=(seq_length, nb_features)),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Softmax()
])

# -- Compilation du réseau --
model_lstm.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model_lstm.summary()

# -- Entrainement du model --
history = model_lstm.fit(xTrain, yTrain, epochs=epochs, batch_size=batch_size, verbose=verbose)

# -- Evaluation du model --
model_lstm.evaluate(xTest, yTest)

# -- Sauvegarde du model --
model_lstm.save('/content/drive/MyDrive/model_lstm')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_1 (Masking)         (None, 440, 24)           0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                7296      
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                                                 
 softmax_1 (Softmax)         (None, 6)                 0         
                                                                 
Total params: 7,494
Trainable params: 7,494
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
73/73 [==============================] - 24s 244ms/step - loss: 1.3713 - accuracy: 0.4354
Epoch 2/50
73/73 [==============================] - 18s 252ms/step - loss: 0.9683 -

In [47]:
# -- Conversion du model en TFLite --
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file('/content/drive/MyDrive/model_lstm')
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.experimental_lower_tensor_list_ops = False
tflite_model_lstm = converter.convert()
with open('model_lstm.tflite', 'wb') as f:
  f.write(tflite_model_lstm)



# -- Test du TFLite --
nb_test = 10
interpreter = tf.lite.Interpreter(model_path='/content/model_lstm.tflite')
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

for i in range(nb_test):
  input_data = np.random.random((1, seq_length, nb_features)).astype(np.float32)
  interpreter.set_tensor(input_index, input_data)
  expected = model_lstm(input_data)
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_index)
  np.testing.assert_almost_equal(expected, output_data, decimal=4)
  print(expected)
  print(output_data)
interpreter.reset_all_variables()

Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' to decorate the function.
Use '@tf.function' or '@defun' 

tf.Tensor([[0.146936   0.68780434 0.02417044 0.02417044 0.09274833 0.02417044]], shape=(1, 6), dtype=float32)
[[0.14693606 0.6878043  0.02417043 0.02417043 0.09274832 0.02417043]]
tf.Tensor([[0.20928006 0.4754958  0.07576489 0.03704979 0.16535963 0.03704979]], shape=(1, 6), dtype=float32)
[[0.20928006 0.47549567 0.07576489 0.0370498  0.16535972 0.0370498 ]]
tf.Tensor([[0.08783381 0.85865426 0.00812486 0.00812486 0.02913726 0.00812486]], shape=(1, 6), dtype=float32)
[[0.08783385 0.85865426 0.00812487 0.00812487 0.02913729 0.00812487]]
tf.Tensor([[0.09390693 0.86612827 0.00783732 0.00783732 0.01645279 0.00783732]], shape=(1, 6), dtype=float32)
[[0.09390689 0.8661283  0.00783732 0.00783732 0.01645279 0.00783732]]
tf.Tensor([[0.1204162  0.7917067  0.01477532 0.01477532 0.04355128 0.01477532]], shape=(1, 6), dtype=float32)
[[0.12041622 0.7917064  0.01477533 0.01477533 0.04355127 0.01477533]]
tf.Tensor([[0.08999602 0.8350449  0.00922859 0.00922859 0.04727321 0.00922859]], shape=(1, 6), dtype

In [18]:
files = ["/content/drive/MyDrive/model_lstm/keras_metadata.pb"]
populator = _metadata.MetadataPopulator.with_model_file('/content/model_lstm.tflite')
populator.load_associated_files(files)
populator.populate()

/usr/local/lib/python3.10/dist-packages/tflite_support/metadata.py:273: UserWarning: File, 'keras_metadata.pb', does not exsit in the metadata. But packing it to tflite model is still allowed.
  warnings.warn(


In [44]:
# Creates model info.
model_meta = _metadata_fb.ModelMetadataT()
model_meta.name = "ReferAI"
model_meta.description = "LSTM Model for Powerlift arbitration"
model_meta.version = "v1"
model_meta.author = "Crevel Tristan"

# Creates input info.
input_meta = _metadata_fb.TensorMetadataT()
input_meta.description = ("Array of 440*24")


# Creates output info.
output_meta = _metadata_fb.TensorMetadataT()
output_meta.name = "probability"
output_meta.descritpion = 'Probability of the 6 labels'


# Creates subgraph info.
subgraph = _metadata_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(model_meta.Pack(b), _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

populator = _metadata.MetadataPopulator.with_model_file("model_lstm.tflite")
populator.load_metadata_buffer(metadata_buf)
populator.populate()
